In [1]:
from datetime import datetime
from datetime import timedelta
import yfinance as yf
import time
import math
import pandas as pd
import pandas_ta as ta
import json
import numpy as np
import pandas as pd
import http.client
import json
#casove pasma
import pytz
import os

In [2]:
#get current date and time
tz_BA = pytz.timezone('Europe/Bratislava')
current_datetime = datetime.now(tz_BA).strftime("%Y-%m-%d %H-%M-%S")
# convert datetime obj to string
str_current_datetime = str(current_datetime)
# create a file name
file_name = "//home//jupyter-fiko//sesions log//"+"sesionBTCETH "+"session_breakout "+str_current_datetime+".txt"

#We can adjust Jupyter with just a pinch of code so that it saved the output directly to a file on the server
import sys
import logging
so = open(file_name, 'w', 10)
sys.stdout.echo = so
sys.stderr.echo = so
get_ipython().log.handlers[0].stream = so
get_ipython().log.setLevel(logging.INFO)

FileNotFoundError: [Errno 2] No such file or directory: '//home//jupyter-fiko//sesions log//sesionBTCETH session_breakout 2023-12-29 21-04-05.txt'

In [ ]:
%run /home/jupyter-fiko/bot_functions.ipynb
%run /home/jupyter-fiko/capital_commands.ipynb

In [ ]:
asset_list = [
    {'ticker':'spx500', 'yfticker':yf.Ticker('^GSPC'),'capitalticker':'US500'},
    {'ticker':'nasdaq', 'yfticker':yf.Ticker('^NDX'),'capitalticker':'US100'},
    {'ticker':'gold', 'yfticker':yf.Ticker('gdx'),'capitalticker':'GOLD'},
    {'ticker':'BAC', 'yfticker':yf.Ticker('BAC'),'capitalticker':'BAC'},
    {'ticker':'AAPL', 'yfticker':yf.Ticker('AAPL'),'capitalticker':'AAPL'}
]

In [ ]:
#print_acount_ids(keys)
#190868445419885726 = tradingbot#204592811554976030 = Breakout strategy 11k start

In [ ]:
keys =get_demo_keys()

In [ ]:
position_size = 0.1
   
while True:
    ping(keys)
    #choose trading acount
    switch_acount(keys,204592811554976030)
    #with wrong name u get error
    balance =  acount_balance(keys,'Breakout strategy 11k start')
    
    for asset in asset_list:
        start_date = (datetime.now()-timedelta(days=2)).strftime('%Y-%m-%d')
        df = asset['yfticker'].history(start=start_date, interval='30m')
        price = df.iloc[-1]['Close']
        leverage = instrument_leverage(keys,asset['capitalticker'])
        position = round((position_size * balance * leverage) / price,2)
        generate_Breakout_data(df)
        tz_BA = pytz.timezone('Europe/Bratislava')
        BA = datetime.now(tz_BA)

        # Defining breakout conditions
        # Conditions:
        # 1. Green candle,
        # 2. Candle's body is longest in the last 10 days,
        # 3. Breakout volume is 50% higher than the rolling 20-day average, and
        # 4. Breakout candle has a body that is 100% higher than the rolling 20-day average
        condition = (
            (df['O-to-C'] >= 0.0) & 
            (df['O-to-C'] == df['MaxOC_Prev10']) & 
            (df['OC-%-from-20D-Mean'] >= 100.0) & 
            (df['Volume-%-from-20D-Mean'] >= 50.0))
        #aditional condition is avoinding buying at last hours of trading day
        if condition.iloc[-1] == True and df.iloc[-1,0].hour != 15 and df.iloc[-1,0].minute != 30:
            print(f"Buy {asset['ticker']} at {price} $")
                
            open_position_with_stoploss(keys, asset['capitalticker'], 
            "BUY", position, stoplosslevel= (df.iloc[-1]['Close'] - df.iloc[-1]['Open']) , trailstop= True     )
            
        print( asset['ticker']+" checked " +", time =  ",BA.strftime("%H:%M:%S") )
    print(f"Sleep for {get_30min_pause()} seconds")
    ping_server_till_next_30min(keys)
    time.sleep(get_30min_pause())